# Optimization of an IoV system

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Importing the data

In [58]:
rsu = pd.read_csv('data/rsu.csv')
es = pd.read_csv('data/es.csv')
tasks = pd.read_csv('data/tasks.csv')

# Generating a Network

In [62]:
def get_network(rsu: pd.DataFrame, es: pd.DataFrame):
    # choose a random number of RSUs M
    M = np.random.randint(1, len(rsu))
    # choose a random number of ESs W, with W < M
    W = np.random.randint(1, M)

    # choose M random RSUs
    rsu = rsu.sample(M)
    # choose W random ESs
    es = es.sample(W)

    # append empty rows to the es dataframe for it to have the same number of rows as rsu
    # use pandas' concat function to append the empty rows
    es = pd.concat([es, pd.DataFrame(columns=es.columns)], ignore_index=True)

    # shuffle the datasets
    rsu = rsu.sample(frac=1).reset_index(drop=True)
    es = es.sample(frac=1).reset_index(drop=True)

    # merge the rsu and es dataframes 
    rsu = pd.concat([rsu, es], axis=1)

    return {
        'M': M,
        'W': W,
        'network': rsu,
    }


In [63]:
network = get_network(rsu, es)

In [64]:
print(f"Number of RSUs: {network['M']}")
print(f"Number of ESs: {network['W']}")

Number of RSUs: 81
Number of ESs: 4


In [66]:
print(network['network'])

    id   x   y  dtr   id  vms   cp
0   86  56  70   30   89    3    7
1   33  59  21   34   40    1    6
2   43  19   2    6   56    4    9
3   82  80  51    1   92    3    7
4   87  89  89   47  NaN  NaN  NaN
..  ..  ..  ..  ...  ...  ...  ...
76  39  25  94    3  NaN  NaN  NaN
77  79  36   9   18  NaN  NaN  NaN
78  13  52  23   32  NaN  NaN  NaN
79  92  95  56   26  NaN  NaN  NaN
80  67  51   8   53  NaN  NaN  NaN

[81 rows x 7 columns]
